In [ ]:
#проверка числа памяти, которая задейстована
!nvidia-smi

Wed Mar  5 09:38:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%pip install  transformers==4.47.1 accelerate==1.2.1 sentencepiece==0.2.0 optimum==1.23.3 auto-gptq==0.7.1 torchmetrics
%pip install -U bitsandbytes
import gc
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import bitsandbytes as bnb
import subprocess
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
from IPython.display import clear_output
from random import sample
from tqdm import tqdm
from torchmetrics.functional import accuracy
from torch.optim import AdamW
from peft import PromptTuningConfig, PromptTuningInit, get_peft_model, LoraConfig, TaskType
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.6/931.6 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 852.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR:bitsandbytes.cextension:Could not load bitsandbytes native library: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.32' not found (required by /usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cpu.so)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 85, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 72, in get_native_library
    dll = ct.cdll.LoadLibrary(str(binary_path))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 454, in LoadLibrary
    return self._dlltype(name)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4

In [ ]:
!pip install evaluate
!pip install rouge_score
!pip install sacrebleu
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ce32370b97746709b19a85c566d0e3dadd9ed4ef01bec59dee4fc7f9fba207ee
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
import json
import re
import ast
from typing import List, Dict, Any

def parse_all_data(file_path: str) -> List[Dict[str, Any]]:
    """Парсинг всех данных без учета времени ответа"""
    return _parse_data(file_path, include_time=False)

def parse_data_with_time(file_path: str) -> List[Dict[str, Any]]:
    """Парсинг данных с сохранением времени ответа"""
    return _parse_data(file_path, include_time=True)

def _parse_data(file_path: str, include_time: bool) -> List[Dict[str, Any]]:
    """Базовая функция парсинга"""
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    result = []
    for item in data:
        parsed = {
            'selected_role': item['Выбранная роль'],
            'campus': item['Кампус'],
            'education_level': item['Уровень образования'],
            'question_category': item['Категория вопроса'],
            'user_question': _clean_text(item['Вопрос пользователя']),
            'user_filters': item['user_filters'],
            'question_filters': item['question_filters'],
            'saiga_answer': _clean_text(item['Saiga']),
            'giga_answer': _clean_text(item['Giga']),
            'winner': item['Кто лучше?'],
            'comment': item['Комментарий'],
            'contexts': _parse_contexts(item['Ресурсы для ответа'])
        }

        if item.get('Уточненный вопрос пользователя'):
            parsed.update({
                'refined_question': _clean_text(item['Уточненный вопрос пользователя']),
                'refined_answer': _clean_text(item['Ответ AI (уточнение)']),
                'refined_contexts': _parse_contexts(item['Ресурсы для ответа (уточнение)'] or '')
            })

        if include_time:
            parsed.update({
                'response_time': item['Время ответа модели (сек)'],
                'refined_response_time': item.get('Время ответа модели на уточненный вопрос (сек)')
            })

        result.append(parsed)

    return result

def _parse_contexts(resources: str) -> List[Dict[str, Any]]:
    """Парсинг контекстов с использованием вашей функции"""
    contexts = []
    pattern = re.compile(r"Document\(page_content='(.*?)', metadata=({.*?})\)", re.DOTALL)

    for match in re.finditer(pattern, resources):
        content, metadata_str = match.groups()
        try:
            metadata = ast.literal_eval(metadata_str)
            tags = _extract_tags(metadata)

            contexts.append({
                'text': _clean_text(content),
                'metadata': {
                    'source': metadata.get('source'),
                    'file_name': metadata.get('file_name'),
                    'url': metadata.get('url')
                },
                'tags': tags
            })
        except Exception as e:
            print(f"Контекст не распарсился: {e}")

    return contexts

def _extract_tags(metadata: Dict) -> Dict[str, List[str]]:
    """Извлечение тегов в отдельные категории"""
    return {
        'topic_tags': [v for k,v in metadata.items() if k.startswith('topic_tag_') and v],
        'user_tags': [v for k,v in metadata.items() if k.startswith('user_tag_') and v]
    }

def _clean_text(text: str) -> str:
    """Очистка текста"""
    if not text: return ''
    return re.sub(r'\\[nrt]|[\n\r\t]+|\s+', ' ', text).strip()

In [ ]:
data_v1 = parse_all_data('train_set.json')

data_v2 = parse_all_data('val_set.json')

with open('parsed_tuning.json', 'w', encoding='utf-8') as f:
    json.dump(data_v1, f, ensure_ascii=False)

with open('parsed_dash.json', 'w', encoding='utf-8') as f:
    json.dump(data_v2, f, ensure_ascii=False)

In [ ]:
import json

with open('parsed_tuning.json', 'r', encoding='utf-8') as f:
    training_data = json.load(f)

In [ ]:
formatted_data = []
for item in training_data:
    contexts = "\n".join([ctx['text'] for ctx in item['contexts']])
    base_input = f"Вопрос: {item['user_question']}\nКонтекст: {contexts}"

    if item['winner'] == 'Saiga':
        formatted_data.append({
            "input": base_input,
            "output": item['saiga_answer'],
            "source": "saiga",
            "rating": "good" if item['winner'] in ['Saiga', 'Оба хорошо'] else "bad"
        })

    elif item['winner'] == 'GigaChat':
        formatted_data.append({
            "input": base_input,
            "output": item['giga_answer'],
            "source": "giga",
            "rating": "good" if item['winner'] in ['GigaChat', 'Оба хорошо'] else "bad"
        })

    elif item['winner'] == 'Оба хорошо':
        formatted_data.extend([
            {
                "input": base_input,
                "output": item['saiga_answer'],
                "source": "saiga",
                "rating": "good"
            },
            {
                "input": base_input,
                "output": item['giga_answer'],
                "source": "giga",
                "rating": "good"
            }
        ])

    elif item['winner'] == 'Оба плохо':
        formatted_data.extend([
            {
                "input": base_input,
                "output": item['saiga_answer'],
                "source": "saiga",
                "rating": "bad"
            },
            {
                "input": base_input,
                "output": item['giga_answer'],
                "source": "giga",
                "rating": "bad"
            }
        ])

    else:
        formatted_data.append({
            "input": base_input,
            "output": item['saiga_answer'],
            "source": "unknown",
            "rating": "neutral"
        })

In [ ]:
with open('parsed_dash.json', 'r', encoding='utf-8') as f:
    val_data = json.load(f)

In [ ]:
formatted_data_val = []
for item in val_data:
    contexts = "\n".join([ctx['text'] for ctx in item['contexts']])
    base_input = f"Вопрос: {item['user_question']}\nКонтекст: {contexts}"

    if item['winner'] == 'Saiga':
        formatted_data_val.append({
            "input": base_input,
            "output": item['saiga_answer'],
            "source": "saiga",
            "rating": "good" if item['winner'] in ['Saiga', 'Оба хорошо'] else "bad"
        })

    elif item['winner'] == 'GigaChat':
        formatted_data_val.append({
            "input": base_input,
            "output": item['giga_answer'],
            "source": "giga",
            "rating": "good" if item['winner'] in ['GigaChat', 'Оба хорошо'] else "bad"
        })

    elif item['winner'] == 'Оба хорошо':
        formatted_data_val.extend([
            {
                "input": base_input,
                "output": item['saiga_answer'],
                "source": "saiga",
                "rating": "good"
            },
            {
                "input": base_input,
                "output": item['giga_answer'],
                "source": "giga",
                "rating": "good"
            }
        ])

    elif item['winner'] == 'Оба плохо':
        formatted_data_val.extend([
            {
                "input": base_input,
                "output": item['saiga_answer'],
                "source": "saiga",
                "rating": "bad"
            },
            {
                "input": base_input,
                "output": item['giga_answer'],
                "source": "giga",
                "rating": "bad"
            }
        ])

    else:
        formatted_data_val.append({
            "input": base_input,
            "output": item['saiga_answer'],
            "source": "unknown",
            "rating": "neutral"
        })

In [ ]:
with open("system_prompt.yaml", "r") as f:
  system_prompt = ''.join(f.readlines()).split('system_template:')[1].strip()



formatted_data_prompt = []
for item in formatted_data:
    formatted_data_prompt.append({
        "input": f"{system_prompt}\n\n{item['input']}",  # Добавляем системный промпт
        "output": item["output"]
    })

In [ ]:
train_inputs = [elem['input'] for elem in formatted_data if elem["rating"] == "good"]
train_labels = [elem['output'] for elem in formatted_data if elem["rating"] == "good"]

In [ ]:
train_inputs[0]

In [ ]:
val_inputs = [elem['input'] for elem in formatted_data_val]
val_labels = [elem['output'] for elem in formatted_data_val]

In [ ]:
model_name = 'yandex/YandexGPT-5-Lite-8B-pretrain'

tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.,
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    offload_state_dict=True,
    #quantization_config=quantization_config,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.

In [ ]:
def tokenize_data(inputs, labels, tokenizer, max_length=128):
    input_encodings = tokenizer(
        list(inputs), max_length=max_length, padding=True, truncation=True, return_tensors="pt"
    )
    label_encodings = tokenizer(
        list(labels), max_length=max_length, padding=True, truncation=True, return_tensors="pt"
    )
    return input_encodings, label_encodings

train_inputs_enc, train_labels_enc = tokenize_data(train_inputs, train_labels, tokenizer)
test_inputs_enc, test_labels_enc = tokenize_data(val_inputs, val_labels, tokenizer)

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels["input_ids"][idx],
        }

train_dataset = CustomDataset(train_inputs_enc, train_labels_enc)
test_dataset = CustomDataset(test_inputs_enc, test_labels_enc)

train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=3)

In [ ]:
def train(model, optimizer, train_loader):
  model.train()
  epochs = 4
  for epoch in range(epochs):
      for batch in train_loader:
          optimizer.zero_grad()

          input_ids = batch["input_ids"].to(device)
          attention_mask = batch["attention_mask"].to(device)
          labels = batch["labels"].to(device)

          outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
          loss = outputs.loss
          loss.backward()
          optimizer.step()

      print(f"Epoch {epoch + 1} Loss: {loss.item()}")

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # Задача генерации текста
    inference_mode=False,          # Режим вывода (False для обучения)
    r=8,                           # Ранг адаптации
    lora_alpha=16,                 # Масштабирующий коэффициент
    lora_dropout=0.1,              # Dropout для LoRA
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj']     # Целевые модули для адаптации
)


In [ ]:
lora_model = get_peft_model(model, peft_config)
device = "cuda" if torch.cuda.is_available() else "cpu"
lora_model.to_empty(device=device)  # Перемещение модели на устройство
#lora_model.load_state_dict(torch.load("path-to-your-weights.pt"))  # Загрузка весов
lora_model.print_trainable_parameters()
"""    "togethercomputer/RedPajama-INCITE-3B-v1",
"""

trainable params: 6,815,744 || all params: 8,043,368,448 || trainable%: 0.0847


'    "togethercomputer/RedPajama-INCITE-3B-v1",\n'

In [ ]:
from peft import PromptTuningConfig, PromptTuningInit, get_peft_model, LoraConfig, TaskType
init = {PromptTuningInit.TEXT: """Ты - система информационного поиска Национального исследовательского университета «Высшая школа экономики» (НИУ ВШЭ). Тебе дан вопрос и релевантные отрывки текста из нескольких документов.
Создай краткий и информативный ответ на заданный вопрос. Ты должна использовать только информацию из приведенных отрывков.
Используй непредвзятый и журналистский тон. Не повторяй текст. Не пытайся придумать ответ.
Если в контексте присутствуют ссылки на документы, выведи ответ со ссылками на источники.
Создай окончательный ответ ("FINAL ANSWER").
Отвечай только на русском языке за исключением специальных терминов.
ПРИМЕР:
QUESTION: Что такое ИУП?
=========
Content 1: Индивидуальный учебный план (ИУП) — документ студента, в соответствии с которым он обязуется освоить элементы обучения (дисциплин/элементов практической подготовки) в определенный период.
По каждому элементу ИУПа хранится информация
Content 2: период.
По каждому элементу ИУПа хранится информация о технологиях обучения, сроках изучения и прохождения промежуточной аттестации, объеме учебной нагрузки и основании включения этого элемента в учебный план.
Для студентов большинства
Content 3: план.
Для студентов большинства образовательных программ, ИУП составляется на полугодие, и все элементы, включенные в ИУП, обязательны для освоения в этот период.
Как ознакомиться со своим ИУП
Content 4: со своим ИУП. Ваш ИУП доступен в [электронной зачетке]( https://lms.hse.ru/index.php?page=gradebook) LMS НИУ ВШЭ. \nПодробнее об ИУП: https://www.hse.ru/studyspravka/plan
=========
FINAL ANSWER: Индивидуальный учебный план (ИУП) — документ студента, в соответствии с которым он обязуется освоить элементы обучения (дисциплин/элементов практической подготовки) в определенный период. \nИУП доступен в электронной зачетке LMS НИУ ВШЭ (https://lms.hse.ru/index.php?page=gradebook). \nПодробнее об ИУП: https://www.hse.ru/studyspravka/plan

Если вопрос остался непонятен или слишком широкий, веди диалог с пользователем и попроси уточнить необходимую информацию: "Пожалуйста, уточните ...". Проявляй эмпатию и поддержку.
QUESTION: Я боюсь, что меня отчислят...
=========
Content 1: отчисления по данному основанию допускается, если у обучающегося отсутствует возможность продолжать обучение в университете в связи с возникшими обстоятельствами)
Content 2: Для профессиональной помощи в Дирекции сопровождения отдельных категорий студентов работает психолог[Болдырева Наталья Анатольевна](https://www.hse.ru/org/persons/902647165) . Также в любое время можете получить консультацию специалистов[Центра психологического консультирования](https://www.hse.ru/cpc/), заполнив[онлайн-формудля записи на консультацию к психологу](https://bpm.hse.ru/Runtime/Runtime/Form/RPC__f__NewInsideRequest/?&usertype=1) на сайтеили позвонив потелефону 8(915) 260-06-20.
Content 3: Студенты, КУД которых превышает критическое значение (с учетом отнесения студента к отдельной категории и наличия или отсутствия повтора критической ситуации), подлежат отчислению как не выполняющие обязанности по добросовестному освоению образовательной программы и выполнению учебного плана.
=========
FINAL ANSWER: Я понимаю ваши опасения. Пожалуйста, уточните в чем причина ваших опасений? Вас беспокоит успеваемость, проблемы с оплатой или другие вопросы? В НИУ ВШЭ также работают специалисты, которые могут оказать вам психологическую помощь. Могу поделиться необходимыми контактами Важно своевременно обращаться за помощью и поддержкой, чтобы избежать возможных проблем с обучением.

Если в отрывках текста не содержится ответ на вопрос, то выведи релевантную информацию, которую ты смог найти. Избегай упоминания дат и иных цифр. Напиши: "Я могу частично ответить на данный вопрос: ... Для получения более релевантной информации обратитесь в учебный офис своей программы."

ПРИМЕР:
QUESTION: Как получить стипендию Потанина?
=========
Content 1: отвечающие критериям, установленным в пункте 2.3 Положения, подают заявку в электронном видев срок до 15 декабря текущего календарного года путем заполнения электронной формы в личном кабинете аспиранта на корпоративном сайте (портале) НИУ ВШЭ (http://www.hse.ru/user). Для подачи заявки аспирант предварительно загружает публикации в личном кабинете на корпоративном сайте (портале) НИУ ВШЭ. Заявки на стипендию принимаются ежегодно с 15 ноября по 15 декабря.
Content 2: Положение о назначении и выплате именной стипендии имени Е.Т. Гайдара аспирантам Аспирантской школы по экономике Национального исследовательского университета «Высшая школа экономики»
=========
FINAL ANSWER: Я нашла информацию о том, что на некоторые стипендии заявку можно подать в электронном виде через личный кабинет на корпоративном сайте НИУ ВШЭ. Однако, я не могу сориентировать о процессе подачи на стипендию Потанина. Для получения более релевантной информации обратитесь в учебный офис своей программы.

Если вопрос пользователя не соответсвует тематике вопросов обучения НИУ ВШЭ, не этичен или содержит просьбы о помощи с домашним заданием, написанием ВКР и др, ответь следующим образом:
"Мне кажется неуместно здесь это обсуждать. Моя цель - проконсультировать тебя по вопросам обучения. Может, у тебя есть еще какие-то вопросы?"
"""}
config_lora = lora_model.config


pt_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=init,
    num_virtual_tokens=13
)

pt_model = get_peft_model(lora_model, pt_config)
pt_model.print_trainable_parameters()

trainable params: 53,248 || all params: 8,043,421,696 || trainable%: 0.0007


In [ ]:
optimizer = AdamW(pt_model.parameters(), lr=5e-4)

In [ ]:
train(pt_model, optimizer, train_loader)

In [ ]:
pt_model.save_pretrained("./my_trained_model")  # Сохраняет веса и конфигурацию
tokenizer.save_pretrained("./my_trained_model")

In [ ]:
import pandas as pd

questions = []
contexts = []
ground_truths = []

for item in formatted_val:
    question = item['input'].split("Вопрос:")[1].split("\nКонтекст:")[0].strip()
    questions.append(question)

    context = item['input'].split("\nКонтекст:")[1].strip()
    contexts.append([context])

    ground_truth = item['output']
    ground_truths.append(ground_truth)


# Создание DataFrame
dataset = pd.DataFrame({
    "question": questions,
    "answer": new_outputs, #["Я ГЛУПИ МОДЕЛЬ НЕ УМЕЮ..."] * len(questions),  # Заглушка для ответов модели
    "ground_truth": ground_truths,
    "contexts": contexts,
})

print(dataset)

In [ ]:
# Загрузка модели и токенизатора
model = AutoModelForCausalLM.from_pretrained("./my_trained_model")
tokenizer = AutoTokenizer.from_pretrained("./my_trained_model")

# Убедитесь, что модель в режиме оценки
model.eval()

#first
def generate_answer(model, tokenizer, input_text, max_length=512):
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=max_length)
    outputs = model.generate(**inputs, max_length=max_length, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)



new_outputs = []
for item in val_data:
    input_text = item["input"]
    prediction = generate_answer(model, tokenizer, input_text)
    new_outputs.append(prediction)


#second
new_outputs = []

# Проход по каждому вопросу в валидационном датасете
for question in validation_data['input']:
    # Токенизация вопроса
    inputs = tokenizer(question, return_tensors="pt")

    # Генерация ответа
    with torch.no_grad():  # Отключаем градиенты для экономии памяти
        outputs = model.generate(**inputs)  # Укажите max_length по необходимости

    # Декодирование ответа
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Добавление ответа в список
    new_outputs.append(answer)

# STOP

In [ ]:
def eval(model, test_loader):
  model.eval()
  for batch in test_loader:
      input_ids = batch["input_ids"].to(device)
      attention_mask = batch["attention_mask"].to(device)
      labels = batch["labels"].to(device)

      input_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in input_ids]
      true_labels = [tokenizer.decode(label, skip_special_tokens=True) for label in labels]

      outputs = model.generate(
          input_ids=input_ids,
          attention_mask=attention_mask,
          max_length=50
      )
      predictions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

      for input_text, true_label, pred in zip(input_texts, true_labels, predictions):
          print("-" * 50)
          print(f"input_txt: {input_text}")
          print(f"true_label: {true_label}")
          print(f"true_pred: {pred}")

      break

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from peft import PromptTuningConfig, get_peft_model

# Загрузка токенизатора и модели GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

config = model.config
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False, num_virtual_tokens=13,
    num_layers=config.n_layer, token_dim=config.n_embd, num_attention_heads=config.n_head,
    prompt_tuning_init=init
)

pt_model = get_peft_model(model, peft_config)

# Перевод модели в режим оценки
pt_model.eval()

def generate_response(input_text):
    # Подготовка входного текста
    input_with_prompt = peft_config.prompt + input_text
    inputs = tokenizer.encode(input_with_prompt, return_tensors="pt")

    # Генерация ответа
    with torch.no_grad():
        outputs = pt_model.generate(inputs, max_length=100, num_return_sequences=1)

    # Декодирование и возвращение ответа
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(peft_config.prompt):]  # Убираем системный промпт

# Пример использования
user_input = "Как мне справиться со стрессом на работе?"
response = generate_response(user_input)
print("Ответ:", response)


In [ ]:
train(pt_model, optimizer, train_loader)

Epoch 1 Loss: 5.05859375
Epoch 2 Loss: 4.640625
Epoch 3 Loss: 4.67578125
Epoch 4 Loss: 4.62890625
Epoch 5 Loss: 5.2421875


In [ ]:
# Удаление переменных
del model
del lora_model
del optimizer
del train_loader
# Освобождение памяти GPU (для PyTorch)
torch.cuda.empty_cache()

In [ ]:
torch.save(lora_model.base_model.model.lm_head.state_dict(), 'lora_head.pt')

In [ ]:
chkpt = torch.load('lora_head.pt')
chkpt.keys()

<ipython-input-30-4c71755dc0ab>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  chkpt = torch.load('lora_head.pt')


odict_keys(['weight'])

In [ ]:
state_dict = torch.load('lora_head.pt')
print(state_dict[''])

<ipython-input-34-fc7b80294d13>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('lora_head.pt')


odict_keys(['weight'])


In [ ]:

model.lm_head.load_state_dict({
    'weight': torch.load('lora_head.pt')['weight']
})
optimizer = AdamW(model.parameters(), lr=2e-6)

<ipython-input-35-9a7c70f25b48>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  'weight': torch.load('lora_head.pt')['weight']


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.02 GiB. GPU 0 has a total capacity of 14.74 GiB of which 354.12 MiB is free. Process 5925 has 14.39 GiB memory in use. Of the allocated memory 13.83 GiB is allocated by PyTorch, and 434.67 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
train(model, optimizer, train_loader)